In [47]:
import sys
sys.path.append('/project/Development')
from helpers import output, xml_helpers, general_helpers
import configparser
config = configparser.ConfigParser()
config.read('/project/Development/config.ini')

['/project/Development/config.ini']

In [48]:
print('test')

test


In [49]:
engine = general_helpers.connect_to_db(config['DATABASE']['HOST'], config['DATABASE']['USERNAME'], config['DATABASE']['PASSWORD'], config['DATABASE']['NEW_DB'])
db_con = engine.connect() 

In [53]:
data = db_con.execute('show processlist;')
for row in data:
    if row[1] == 'skelley':
        print(row)

(1990, 'skelley', '216.59.102.42:34458', None, 'Sleep', 33, '', None, 0.0)
(1991, 'skelley', '216.59.102.42:19919', None, 'Sleep', 33, '', None, 0.0)
(1996, 'skelley', '172.31.8.197:53134', 'patent_20181127', 'Query', 0, 'Init', 'show processlist', 0.0)


In [ ]:
alter table  patent_20181127.temp_uspatentcitation_without_new rename patent_20181127.uspatentcitation;

alter table patent_20181127.uspatentcitation rename patent_20181127.uspatentcitation_feb15_backup;

In [55]:
db_con.execute('ALTER TABLE patent_20181127.patent_govintorg CONVERT TO CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci')

In [54]:
db_con.execute('alter table patent_20181127.uspatentcitation rename patent_20181127.uspatentcitation_feb15_backup;')

OperationalError: (MySQLdb._exceptions.OperationalError) (2013, 'Lost connection to MySQL server during query') [SQL: 'alter table patent_20181127.uspatentcitation rename patent_20181127.uspatentcitation_feb15_backup;'] (Background on this error at: http://sqlalche.me/e/e3q8)

In [16]:
db_con.execute('CREATE TABLE `PatentsView_20181127`.`uspatentcitation_redo` (`citing_patent_id` VARCHAR(20) NOT NULL, `sequence` INT NOT NULL, `cited_patent_id` VARCHAR(20) NULL, `category` VARCHAR(20) NULL, PRIMARY KEY(`citing_patent_id`, `sequence`)) ENGINE = INNODB;')

In [6]:
#db_con.execute("create table patent_20181127.temp_0528_lookup as select id from patent_20181127.patent where date  > '2018-05-28' and date < '2018-11-27';")

In [18]:
#db_con.execute('create index id_ix on patent_20181127.temp_0528_lookup (id)')

In [19]:
db_con.execute('create index patent_ix on patent_20181127.temp_uspatentcitation_without_new (patent_id)')


In [26]:
data = db_con.execute('select patent_id, uuid, sequence from patent_20181127.temp_0528_lookup l left join patent_20181127.temp_uspatentcitation_without_new t on l.id = t.patent_id')

In [27]:
#print(len(data))

In [33]:
patent_seq_dict = defaultdict(lambda : defaultdict(lambda: []))
counter = 0
for row in data:
    counter +=1
    if counter%100000 ==0:
        print(counter)
    patent_id = row['patent_id']
    sequence = row['sequence']
    uuid = row['uuid']
    patent_seq_dict[patent_id][sequence].append(uuid)

In [38]:
to_remove = []
for patent_id, seq_dict in patent_seq_dict.items():
    for seq, list_uuids in seq_dict.items():
        if len(list_uuids)>1:
            to_remove.append(list_uuids[1])

In [42]:
to_remove[-2:]

['gxvwoqejb7uum7r2h29cmn18j', 'y2c2vkoeom8muzznjuf11s0cq']

In [40]:
for e, uuid in enumerate(to_remove):
    if e%10000 ==0:
        print(e)
    db_con.execute("delete from patent_20181127.temp_uspatentcitation_without_new where uuid = '{}'".format(uuid))
    


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000


In [41]:
print('test')

test


In [9]:
from collections import defaultdict

In [17]:
for patent in id_list:
    data = db_con.execute('select uuid, sequence from patent_20181127.temp_uspatentcitation_without_new where patent_id = {}'.format(patent))
    seq_count = defaultdict(lambda :[])
    for row in data:
        seq_count[row['sequence']].append(row['uuid'])
    break

In [13]:
#seq_count

defaultdict(<function __main__.<lambda>()>,
            {0: ['nsxh5izaox5fc0vjzexq4uzdc', '9v136bco1t2kh91vyr7ehpapz'],
             1: ['mik89zqbnjn4knbk2urkmfxh5', '7pd2hkn1brjppnzv7majolja2'],
             2: ['gn42gmudxnyt9r3jiofvo94eb', 'fcirtiefo21x4gxtfbwhb885l'],
             3: ['2enr4pmirguzcjsep0js54ip3', 'u0cyroo5kfw18hg7snor1snvz'],
             4: ['yjx7cru3jnqw5tima0n2er380', '4pggtqqlneve3qtwvq1pf4n6p'],
             5: ['60qb8yeasmc2tr5c6c01abb6k', 'pmuiq2vosfeoe6dxuelzmwfyd'],
             6: ['3zqjscafl2kndpigygoa5hp5y', 'hkkz1u9bbjg22s0hla1yr1ten'],
             7: ['tuiyytaegqkz9gyv8vuhw5r7k', 'lc158f6x49ye5c38xdruf5ftk'],
             8: ['erywbywvavrh4mr6rx0780roc', 'mhxy7t01s1f43eowx40nsf34y'],
             9: ['1lnhn8zzw4z35mcukv3axeon1', 'g55e5y9myxptlc9kie76xs5cj'],
             10: ['82yiqvz6ij0zv0dafcwlqk7bg'],
             11: ['vk8udku5nqz9im26fn5us548p', '5ib1s8b77zzd5uiz4x5sv8sk7'],
             12: ['l7e52ueg4ykj9q8e1nlz8kwgs', 'uaq66kls0he8xmmuoqp0dgu

In [43]:
db_con.execute('DROP TABLE `PatentsView_20181127`.`uspatentcitation_redo`')

In [44]:
db_con.execute('CREATE TABLE `PatentsView_20181127`.`uspatentcitation_redo` (`citing_patent_id` VARCHAR(20) NOT NULL, `sequence` INT NOT NULL, `cited_patent_id` VARCHAR(20) NULL, `category` VARCHAR(20) NULL, PRIMARY KEY(`citing_patent_id`, `sequence`)) ENGINE = INNODB;')

In [45]:
db_con.execute("INSERT INTO `PatentsView_20181127`.`uspatentcitation_redo` (`citing_patent_id`, `sequence`,`cited_patent_id`,`category`) SELECT pc.`patent_id`, pc.`sequence`, nullif(trim(pc.`citation_id`), ''), nullif(trim(pc.`category`), '') FROM `PatentsView_20181127`.`patent` p INNER JOIN patent_20181127.temp_uspatentcitation_without_new pc ON pc.`patent_id` = p.`patent_id`;")